In [36]:
from itertools import combinations
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import re
import json
import copy
import nltk
from google import genai
import ollama
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize

In [19]:
datasets = []
publications = []
tools = []
with open ('./data/datasets.json') as f:
    datasets = json.load(f)
with open ('./data/publications.json') as f:
    publications = json.load(f)
with open ('./data/tools.json') as f:
    tools = json.load(f)

len(datasets), len(publications), len(tools)

(1059, 3199, 243)

In [20]:
titles = [x['metadata']['summary']['title'] for x in datasets]
titles[:5]

['Cystic Fibrosis Patient Microbiology Cultures',
 'Cystic Fibrosis Patient Liver Enzyme',
 'Cystic Fibrosis Patient Annual Review Encounters',
 'Cystic Fibrosis Patient Transplants',
 'Cystic Fibrosis Patient Sweat Tests']

In [21]:
descriptions = [x['metadata']['summary']['description'] for x in datasets]
descriptions[:5]

["The UK CF Registry is a centralised database of all 60 CF centres across the UK. Data are manually entered in calendar years by CF clinical teams for the 99% of people with a diagnosis of CF who consent to their data being donated to the Registry. Data are entered onto a secure web-portal. For more information please see www.cysticfibrosis.org.uk/registry and 'Data Resource Profile: The UK CF Registry' published in the International Journal of Epidemiology (2018 Feb 1;47(1)9-10e).",
 "The UK CF Registry is a centralised database of all 60 CF centres across the UK. Data are manually entered in calendar years by CF clinical teams for the 99% of people with a diagnosis of CF who consent to their data being donated to the Registry. Data are entered onto a secure web-portal. For more information please see www.cysticfibrosis.org.uk/registry and 'Data Resource Profile: The UK CF Registry' published in the International Journal of Epidemiology (2018 Feb 1;47(1)9-10e).",
 "The UK CF Registry

In [22]:
paper_abstracts = [x['abstract'] for x in publications]
paper_abstracts[1]
abstract = paper_abstracts[2005]
from IPython.core.display import HTML, display
display(HTML(abstract))

/var/folders/5t/sdyz102s5ls5vj828z703jv00000gp/T/ipykernel_81588/4133367635.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import HTML, display


In [33]:
def can_you_find_a_dataset(description):
    prompt = (
        "Can you find a name of any dataset(s) in the following description from a paper abstract, "
        "give me a comma separated list of names of possible dataset names, if you dont think there is one, "
        "dont make up any and return no response:\n\n" + description
    )
    
    response = ollama.chat(
        model="llama3.2:latest",
        messages=[{"role": "user", "content": prompt}],
    )
    return response

response_text = can_you_find_a_dataset(abstract)
print(response_text)

model='llama3.2:latest' created_at='2025-02-12T16:19:17.179826Z' done=True done_reason='stop' total_duration=877812689 load_duration=31568452 prompt_eval_count=1149 prompt_eval_duration=106000000 eval_count=9 eval_duration=738000000 message=Message(role='assistant', content='OnCovid Registry, OnCovid', images=None, tool_calls=None)


In [ ]:

def extract_dataset_content(response_text: str) -> str:
     # Ensure that the response_text is a string.
    if not isinstance(response_text, str):
        response_text = str(response_text)
    
    # This regex captures any characters between "content='" and the next "'"
    pattern = r"content='(.*?)'"
    match = re.search(pattern, response_text)
    if match:
        return match.group(1)
    return ""

content = extract_dataset_content(response_text)
print(content)

TypeError: expected string or bytes-like object